In [1]:
from tool import *

# 1. 신규 생산계획 불러와 데이터 가공

In [64]:
pp = pd.read_excel(get_filename(), sheet_name=0, skiprows=1)

In [65]:
pp_date = []
for item in pp.columns:
    if type(item) == datetime.datetime:
        pp_date.append(item)

pp = pd.concat([pp.iloc[:, :7], pp[pp_date]], axis=1)
pp.columns = ['Target Week', 'ODF', 'TCL BOM', 'drop3', 'drop4', 'Production Date', 'WO'] + pp_date
pp = pp[['Target Week', 'ODF', 'TCL BOM', 'Production Date', 'WO'] + pp_date]
pp = pp.set_index(['Target Week', 'ODF', 'TCL BOM'])[pp_date].stack().reset_index()
pp.columns = ['Target Week', 'ODF', 'TCL BOM', 'PP Date', 'QTY']
pp = pp[pp['TCL BOM'].notnull()].reset_index(drop=True)
pp

,Target Week,ODF,TCL BOM,PP Date,QTY
0,24/03/04W,DOK7518513,X3-24M205M3-INS0AU,2024-02-26,800.0
1,24/03/04W,DOK7518510,X3-24M205M3-COS0AU,2024-02-26,800.0
2,24/03/04W,DOK7518511,X3-27M205M3-EUS0AU,2024-02-26,5120.0
3,24/03/04W,DOK7518640,X3-24M205M3-AES0AU,2024-02-27,800.0
4,24/03/04W,DOK7518528,X3-22M205M3-AES1AU,2024-02-27,2520.0
...,...,...,...,...,...
211,24/05/12W,NaN,X3-27M205M3-AAS0AU,2024-05-06,500.0
212,24/05/12W,NaN,X3-27M205M3-AES0AU,2024-05-07,4608.0
213,24/05/12W,NaN,X3-27M205M3-INS0AU,2024-05-08,512.0
214,24/05/12W,NaN,X3-27M205M3-UKS0AU,2024-05-08,1280.0


In [66]:
pp['QTY'].sum()

265216.0

In [67]:
pp = pp[pp['QTY'].apply(lambda x:type(x) in [int, float])]
pp['QTY'] = pp['QTY'].astype(int)
pp['Rep PMS'] = pp['TCL BOM'].map(tcl_bom_map)

In [68]:
pp[pp['Rep PMS'].isnull()] # Rep PMS 명의 결측치 확인

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS


In [69]:
pp['Mapping Model.Suffix'] = pp['Rep PMS'].map(make_dict_from_df(sp[['Rep PMS', 'Mapping Model.Suffix']].drop_duplicates())).fillna('TBD')

In [70]:
pp[pp['Mapping Model.Suffix'].isnull()] # Mapping Model 명의 결측치 확인

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Mapping Model.Suffix


In [71]:
pp['Updated_at'] = datetime.datetime(2024,3,4,21,40) # 업데이트 일자

In [72]:
c1 = pp['TCL BOM'] == 'X3-27M205M3-USS0EU' # Costco 2 pack 수량을 outer box 기준으로 재조정

In [73]:
pp[c1] 

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Mapping Model.Suffix,Updated_at
61,24/03/18W,DOK7518705,X3-27M205M3-USS0EU,2024-03-14,1280,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2024-03-04 21:40:00
75,24/03/25W,DOK7518918,X3-27M205M3-USS0EU,2024-03-18,1280,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2024-03-04 21:40:00
140,24/04/15W,DOK7519067,X3-27M205M3-USS0EU,2024-04-10,1280,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2024-03-04 21:40:00
165,24/04/22W,NaN,X3-27M205M3-USS0EU,2024-04-17,1280,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2024-03-04 21:40:00
183,24/04/29W,NaN,X3-27M205M3-USS0EU,2024-04-22,1280,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2024-03-04 21:40:00


In [74]:
pp.loc[c1, 'QTY'] = pp.loc[c1, 'QTY'] / 2

In [75]:
pp.shape

(216, 8)

In [3]:
with open('D:/Data/TCL PP_result.bin', 'rb') as f:
    ppr_db = pickle.load(f)

In [113]:
ppr_db['PP Date'].max()

Timestamp('2024-03-03 00:00:00')

In [76]:
c1 = pp['PP Date'] >= '2024-02-26'
c2 = pp['PP Date'] <= '2024-03-03'

In [92]:
df.loc[(df['PP Date'] >= datetime.date(2024, 2, 26)) & (df['PP Date'] < datetime.date(2024, 3, 4)), 'Updated_at'] = datetime.datetime(2024,3,4,21,40)

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_4012\309181994.py:1: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df.loc[(df['PP Date'] >= datetime.date(2024, 2, 26)) & (df['PP Date'] < datetime.date(2024, 3, 4)), 'Updated_at'] = datetime.datetime(2024,3,4,21,40)


In [84]:
pp[c1 & c2].sort_values(['PP Date', 'Mapping Model.Suffix'])

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Mapping Model.Suffix,Updated_at
0,24/03/04W,DOK7518513,X3-24M205M3-INS0AU,2024-02-26,800,24MR400-BA.CTRRMV,24MR400-B.ATRQ,2024-03-04 21:40:00
1,24/03/04W,DOK7518510,X3-24M205M3-COS0AU,2024-02-26,800,24MR400-BA.CWPRMV,24MR400-B.AWPQ,2024-03-04 21:40:00
2,24/03/04W,DOK7518511,X3-27M205M3-EUS0AU,2024-02-26,5120,27MR400-BB.CEURMV,27MR400-B.AEUQ,2024-03-04 21:40:00
4,24/03/04W,DOK7518528,X3-22M205M3-AES1AU,2024-02-27,2520,22MR410-BB.CMARMV,22MR410-B.AMAQ,2024-03-04 21:40:00
3,24/03/04W,DOK7518640,X3-24M205M3-AES0AU,2024-02-27,800,24MR400-BA.CMARMV,24MR400-B.AMAQ,2024-03-04 21:40:00
5,24/03/04W,DOK7518512,X3-22M205M3-INS1AU,2024-02-28,1680,22MR410-BB.CTRRMV,22MR410-B.ATRQ,2024-03-04 21:40:00
6,24/03/04W,DOK7518514,X3-22M205M3-COS1AU,2024-02-28,1680,22MR410-BB.CWPRMV,22MR410-B.AWPQ,2024-03-04 21:40:00
7,24/03/04W,DOK7518641,X3-22M205M3-TKS1AU,2024-03-01,840,22MR410-BB.CPDRMV,22MR410-B.APDQ,2024-03-04 21:40:00
8,24/03/04W,DOK7518529,X3-22M205M3-MXS1AU,2024-03-01,840,22MR410-BB.CWMRMV,22MR410-B.AWMQ,2024-03-04 21:40:00
10,24/03/04W,DOK7518191,X3-24M205M3-MAS0AU,2024-03-01,1920,24MR400-BA.CTSRMV,24MR400-B.ATSQ,2024-03-04 21:40:00


In [19]:
ppr_db = pd.concat([ppr_db, pp[c1 & c2]]).reset_index(drop=True)

In [3]:
with open('D:Data/TCL PP_result.bin', 'wb') as f: # PP reulst에 저장
    pickle.dump(ppr_db, f)

NameError: name 'ppr_db' is not defined

In [21]:
pp.shape

(216, 8)

In [22]:
pp = pp[pp['PP Date'] >= '2024-03-04'].reset_index(drop=True)

In [23]:
pp.shape

(195, 8)

In [24]:
with open('D:/Data/TCL PP_new.bin', 'rb') as f:
    pp_db = pickle.load(f)

In [25]:
pp_db

,Target Week,Country,ODF,TCL BOM,Mapping Model.Suffix,Rep PMS,Production Date,WO,PP Date,QTY,Updated_at
0,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-19,2200,2023-10-17 00:36:00
1,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-20,200,2023-10-17 00:36:00
2,NaN,NaN,DOK7516364,X3-24M205M3-ARS0AU,24MR400-B.AMIQ,24MR400-BA.CMIRMV,NaT,1440.0,2023-10-20,1440,2023-10-17 00:36:00
3,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-20,400,2023-10-17 00:36:00
4,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-21,2000,2023-10-17 00:36:00
...,...,...,...,...,...,...,...,...,...,...,...
2229,24/04/29W,NaN,NaN,X3-27M205M3-EUS0AU,27MR400-B.AEUQ,27MR400-BB.CEURMV,NaT,NaN,2024-04-23,1536,2024-02-27 21:41:00
2230,24/04/29W,NaN,NaN,X3-27M205M3-JNS0AU,27MR400-B.AJLQ,27MR400-BB.CJLRMV,NaT,NaN,2024-04-23,320,2024-02-27 21:41:00
2231,24/04/29W,NaN,NaN,X3-27M205M3-PHS0AU,27MR400-B.APHQ,27MR400-BB.CPHRMV,NaT,NaN,2024-04-23,200,2024-02-27 21:41:00
2232,24/04/29W,NaN,NaN,X3-27M205M3-THS0AU,27MR400-B.ATMQ,27MR400-BB.CTMRMV,NaT,NaN,2024-04-23,100,2024-02-27 21:41:00


In [26]:
pp_db = pd.concat([pp_db, pp]).reset_index(drop=True)

In [27]:
pp_db.shape

(2429, 11)

In [28]:
sorted(pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].max()]['PP Date'].unique()) # 신규 PP의 생산일자가 맞게 되었는 지 확인

[numpy.datetime64('2024-03-04T00:00:00.000000000'),
 numpy.datetime64('2024-03-05T00:00:00.000000000'),
 numpy.datetime64('2024-03-06T00:00:00.000000000'),
 numpy.datetime64('2024-03-07T00:00:00.000000000'),
 numpy.datetime64('2024-03-08T00:00:00.000000000'),
 numpy.datetime64('2024-03-09T00:00:00.000000000'),
 numpy.datetime64('2024-03-11T00:00:00.000000000'),
 numpy.datetime64('2024-03-12T00:00:00.000000000'),
 numpy.datetime64('2024-03-13T00:00:00.000000000'),
 numpy.datetime64('2024-03-14T00:00:00.000000000'),
 numpy.datetime64('2024-03-15T00:00:00.000000000'),
 numpy.datetime64('2024-03-16T00:00:00.000000000'),
 numpy.datetime64('2024-03-18T00:00:00.000000000'),
 numpy.datetime64('2024-03-19T00:00:00.000000000'),
 numpy.datetime64('2024-03-20T00:00:00.000000000'),
 numpy.datetime64('2024-03-21T00:00:00.000000000'),
 numpy.datetime64('2024-03-22T00:00:00.000000000'),
 numpy.datetime64('2024-03-23T00:00:00.000000000'),
 numpy.datetime64('2024-03-25T00:00:00.000000000'),
 numpy.datet

In [29]:
# 업데이트된 pp_db를 저장
with open('D:/Data/TCL PP_new.bin', 'wb') as f:
    pickle.dump(pp_db, f)

# 3. TCL PP Result 수정

In [2]:
with open('D:/Data/TCL PP_result.bin', 'rb') as f:
    ppr_db = pickle.load(f)

In [3]:
bom = 'X3-22M205M3-IDS0AU'
ppdate = '2023-12-24'
updated_qty = 2016
updated_at = datetime.datetime(2024, 3, 6, 21, 31)

c1 = ppr_db['TCL BOM'] == bom
c2 = ppr_db['PP Date'] == ppdate

In [6]:
ppr_db.loc[c1 & c2]

,Mapping Model.Suffix,Rep PMS,TCL BOM,PP Date,QTY,Updated_at
69,22MR410-B.ATIQ,22MR410-BB.CTIRMV,X3-22M205M3-IDS0AU,2023-12-24,2016,2024-03-06 21:31:00


In [5]:
ppr_db.loc[c1 & c2, ('QTY', 'Updated_at')] = [updated_qty, updated_at]

In [7]:
with open('D:/Data/TCL PP_result.bin', 'wb') as f:
    pickle.dump(ppr_db, f)